In [1]:
# Gerekli kütüphaneler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import panel as pn

In [2]:
# 1. Veri Yükleme
file_path = 'CAR DETAILS FROM CAR DEKHO.csv'
car_data = pd.read_csv(file_path)
car_data.head(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
# 2. Kategorik Değişkenleri 1-0 Formatına Dönüştürme
car_data['fuel'] = car_data['fuel'].apply(lambda x: 1 if x == 'Diesel' else 0)  # Diesel -> 1, Petrol -> 0
car_data['transmission'] = car_data['transmission'].apply(lambda x: 1 if x == 'Automatic' else 0)  # Automatic -> 1, Manual -> 0
car_data.head(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,0,Individual,0,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,0,Individual,0,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,1,Individual,0,First Owner
3,Datsun RediGO T Option,2017,250000,46000,0,Individual,0,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,1,Individual,0,Second Owner


In [4]:
# Owner sütunlarını dönüştürme (True/False => 1/0)
car_data['owner_First Owner'] = car_data['owner'].apply(lambda x: 1 if x == 'First Owner' else 0)
car_data['owner_Second Owner'] = car_data['owner'].apply(lambda x: 1 if x == 'Second Owner' else 0)
car_data['owner_Third Owner'] = car_data['owner'].apply(lambda x: 1 if x == 'Third Owner' else 0)
car_data['owner_Fourth & Above Owner'] = car_data['owner'].apply(lambda x: 1 if x == 'Fourth & Above Owner' else 0)

# Owner'ın orijinal sütununu kaldırma
car_data.drop(columns=['owner'], inplace=True)

car_data.head(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner_First Owner,owner_Second Owner,owner_Third Owner,owner_Fourth & Above Owner
0,Maruti 800 AC,2007,60000,70000,0,Individual,0,1,0,0,0
1,Maruti Wagon R LXI Minor,2007,135000,50000,0,Individual,0,1,0,0,0
2,Hyundai Verna 1.6 SX,2012,600000,100000,1,Individual,0,1,0,0,0
3,Datsun RediGO T Option,2017,250000,46000,0,Individual,0,1,0,0,0
4,Honda Amaze VX i-DTEC,2014,450000,141000,1,Individual,0,0,1,0,0


In [5]:
# seller_type sütununu sayısal formata dönüştürme
car_data['seller_type'] = car_data['seller_type'].apply(lambda x: 1 if x == 'Dealer' else 0)
car_data.head(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner_First Owner,owner_Second Owner,owner_Third Owner,owner_Fourth & Above Owner
0,Maruti 800 AC,2007,60000,70000,0,0,0,1,0,0,0
1,Maruti Wagon R LXI Minor,2007,135000,50000,0,0,0,1,0,0,0
2,Hyundai Verna 1.6 SX,2012,600000,100000,1,0,0,1,0,0,0
3,Datsun RediGO T Option,2017,250000,46000,0,0,0,1,0,0,0
4,Honda Amaze VX i-DTEC,2014,450000,141000,1,0,0,0,1,0,0


In [6]:
# 3. Özellikler ve Hedef Değişken
X = car_data[["year", "km_driven", "fuel", "seller_type", "transmission", 
          "owner_Second Owner", "owner_Third Owner", "owner_Fourth & Above Owner"]]
y = car_data['selling_price']  # Tahmin edilecek hedef değişken

# 4. Eğitim ve Test Verisi Ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# 5. Model Eğitimi için Hiperparametre Optimizasyonu (Grid Search)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [20, 30],
    'min_samples_split': [2],
    'min_samples_leaf': [1]
}

grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# 6. En İyi Model ile Eğitme
best_params = grid_search.best_params_
final_model = RandomForestRegressor(**best_params, random_state=42)
final_model.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


RandomForestRegressor(max_depth=30, random_state=42)

In [8]:
car_data['brand'] = car_data['name'].apply(lambda x: x.split()[0])  # Markayı çıkaralım
brand_target_mean = car_data.groupby('brand')['selling_price'].mean()  # Marka bazında ortalama fiyat


# Target Encoding'i veri setine ekleyelim
car_data['brand_encoded'] = car_data['brand'].map(brand_target_mean)
car_data.head(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner_First Owner,owner_Second Owner,owner_Third Owner,owner_Fourth & Above Owner,brand,brand_encoded
0,Maruti 800 AC,2007,60000,70000,0,0,0,1,0,0,0,Maruti,342416.295312
1,Maruti Wagon R LXI Minor,2007,135000,50000,0,0,0,1,0,0,0,Maruti,342416.295312
2,Hyundai Verna 1.6 SX,2012,600000,100000,1,0,0,1,0,0,0,Hyundai,419940.260658
3,Datsun RediGO T Option,2017,250000,46000,0,0,0,1,0,0,0,Datsun,297026.945946
4,Honda Amaze VX i-DTEC,2014,450000,141000,1,0,0,0,1,0,0,Honda,538146.781746


In [9]:
pn.extension()

# Girişler
year = pn.widgets.IntSlider(name="Yıl", start=2000, end=2024, value=2015)
km_driven = pn.widgets.IntInput(name="Kilometre", value=50000)
fuel = pn.widgets.Select(name="Yakıt Türü", options=['Petrol', 'Diesel'])
transmission = pn.widgets.Select(name="Vites Türü", options=['Manual', 'Automatic'])
owner = pn.widgets.Select(name="Sahiplik", options=['İlk', 'İkinci', 'Üçüncü', 'Daha Fazla'])
brand = pn.widgets.Select(name= "Marka", options=['Maruti', 'Hyundai', 'Datsun', 'Honda', 'Tata', 'Chevrolet', 'Toyota', 
                               'Jaguar', 'Mercedes-Benz', 'Audi', 'Skoda', 'Jeep', 'BMW', 'Mahindra', 
                               'Ford', 'Nissan', 'Renault', 'Fiat', 'Volkswagen', 'Volvo', 'Mitsubishi', 
                               'Land', 'Daewoo', 'MG', 'Force', 'Isuzu', 'OpelCorsa', 'Ambassador', 'Kia'])

# Çıkış
output = pn.pane.Str("")

# Tahmin fonksiyonu
def predict():
    try:
        # Marka kodlaması
        brand_encoded_value = brand_target_mean.get(brand, 0)  # Marka için encoding yapılır
        # Kullanıcı girişlerini işleme
        input_data = pd.DataFrame({
            "year": [year.value],
            "km_driven": [km_driven.value],
            "fuel": [1 if fuel.value == 'Diesel' else 0],
            "transmission": [1 if transmission.value == 'Automatic' else 0],
            "owner_Second Owner": [1 if owner.value == 'İkinci' else 0],
            "owner_Third Owner": [1 if owner.value == 'Üçüncü' else 0],
            "owner_Fourth & Above Owner": [1 if owner.value == 'Daha Fazla' else 0],
            "brand_encoded": [brand_encoded_value]
        })

        # Eğitim sırasında kullanılan sütunlar
        expected_columns = list(X_train.columns)

        # Eksik sütunları tamamla
        for col in expected_columns:
            if col not in input_data.columns:
                input_data[col] = 0

        # Sütun sırasını düzelt
        input_data = input_data[expected_columns]

        # Tahmin
        prediction = final_model.predict(input_data)[0]
        output.object = f"Tahmini Satış Fiyatı: {prediction:.2f} TL"
    except Exception as e:
        output.object = f"Hata: {e}"

button = pn.widgets.Button(name="Tahmini Gör")
button.on_click(lambda event: predict())

# Arayüz
pn.Column(year, brand, km_driven, fuel, transmission, owner, button, output).servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'76d32ef6-19a9-42fe-b850-a97f87498be7': {'version…